# VESC Communication - Fundamentals Introduction

Welcome! This notebook will teach you everything you need to know about **communicating with a VESC** (Vedder Electronic Speed Controller). You'll learn how to read data from the VESC, understand what that data means, and send commands to control the motor.

## What is a VESC?

A VESC is like the "brain" that controls an electric motor. Think of it like this:
- **Battery** = Gas tank (stores energy)
- **Motor** = Engine (creates motion)
- **VESC** = Driver (controls how fast the motor spins)
- **Your Computer** = Passenger giving directions to the driver

The VESC takes power from the battery and controls how much goes to the motor. But here's the cool part: **we can talk to the VESC!** We can ask it questions like "How fast is the motor spinning?" and give it commands like "Speed up to 50%!"

## Important Notes About This Demo

📋 **What You'll See in This Notebook:**
- Most readings will show **0** or **None** when the motor isn't moving
- **Voltage readings will work** - they show the actual battery voltage
- **To see real data:** Manually spin the motor/wheel by hand while running the cells
- **Advanced sensors** (ADC, servo, etc.) are not connected to anything, so they'll show 0

This is normal! The VESC only has interesting data to report when something is happening. It's like asking "How fast is the car going?" when it's parked - the answer is 0 until you start driving!

## Setup - Connecting to the VESC

First, let's connect to our VESC motor controller:

In [ ]:
# Setup Python path to find student_api from parent directory
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

from student_api import VESCStudentAPI
import time

vesc_api = VESCStudentAPI() # Create and start VESC API

if vesc_api.start():
    time.sleep(6)  # Wait for controller discovery
    
    connected_controllers = vesc_api.get_connected_controllers() # Check if VESC controller 74 is connected
    vesc = vesc_api.get_controller(74)
    
    if 74 in connected_controllers and vesc.is_connected():
        voltage = vesc.get_input_voltage()
        print(f"✅ Connected to VESC controller! Battery: {voltage:.1f}V")
    else:
        print("❌ VESC controller not found or not responding! Check that it is powered ON.")
else:
    print("❌ Failed to start VESC system")

## Part 1: Reading Motor Information (Telemetry)

Just like a car dashboard shows you speed, fuel, and temperature, the VESC can tell us lots of information about the motor. Let's learn about each piece of information we can read.

# Get the current motor speed
rpm = vesc.get_rpm()

print(f"Motor Speed: {rpm} RPM")

if rpm == 0:
    print("🛑 Motor is stopped")
    print("💡 Try spinning the motor by hand to see a non-zero value!")
elif rpm > 0:
    print("➡️ Motor is spinning forward")
else:
    print("⬅️ Motor is spinning backward")

In [ ]:
# Get the current motor speed
rpm = vesc.get_rpm()  # Ask the VESC how fast the motor is spinning

print(f"Motor Speed: {rpm} RPM")  # Display the speed in Revolutions Per Minute

if rpm == 0:  # Check if motor is stopped
    print("🛑 Motor is stopped")  # Tell user motor isn't moving
elif rpm > 0:  # Check if motor spins forward
    print("➡️ Motor is spinning forward")  # Tell user direction is forward
else:  # Otherwise motor spins backward
    print("⬅️ Motor is spinning backward")  # Tell user direction is backward

# Get how much electricity the motor is using
current = vesc.get_motor_current()

print(f"Motor Current: {current} A")

if current == 0:
    print("⚡ No power flowing to motor")
    print("💡 Spin the motor by hand to generate some current!")
elif current > 0:
    if current < 5:
        print("💚 Low power - motor working gently")
    elif current < 15:
        print("💛 Medium power - motor working normally")
    else:
        print("❤️ High power - motor working very hard!")
else:
    print("🔋 Motor is generating electricity (like a dynamo on a bike!)")

In [ ]:
# Get how much electricity the motor is using
current = vesc.get_motor_current()  # Ask VESC for motor current in Amperes

print(f"Motor Current: {current} A")  # Display the current

if current == 0:  # Check if no power is flowing
    print("⚡ No power flowing to motor")  # Tell user no electricity is being used
elif current > 0:  # Check if motor is using power
    if current < 5:  # Low power usage
        print("💚 Low power - motor working gently")  # Motor using small amount
    elif current < 15:  # Medium power usage  
        print("💛 Medium power - motor working normally")  # Motor using normal amount
    else:  # High power usage
        print("❤️ High power - motor working very hard!")  # Motor using lots of power
else:  # Negative current means generating electricity
    print("🔋 Motor is generating electricity (like a dynamo on a bike!)")  # Motor acting like generator

# Get the current throttle setting
duty_cycle = vesc.get_duty_cycle()

print(f"Duty Cycle: {duty_cycle}")

if duty_cycle == 0:
    print("🛑 Throttle at 0% - motor should be stopped")
    print("💡 This will be 0 unless we send a command to change it")
elif duty_cycle > 0:
    percentage = duty_cycle * 100
    print(f"➡️ Throttle at {percentage:.1f}% forward")
else:
    percentage = abs(duty_cycle) * 100
    print(f"⬅️ Throttle at {percentage:.1f}% backward")

In [ ]:
# Get the current throttle setting
duty_cycle = vesc.get_duty_cycle()  # Ask VESC what throttle setting is active

print(f"Duty Cycle: {duty_cycle}")  # Display the duty cycle (decimal from -1.0 to 1.0)

if duty_cycle == 0:  # Check if throttle is at zero
    print("🛑 Throttle at 0% - motor should be stopped")  # No throttle applied
elif duty_cycle > 0:  # Check if throttle is forward
    percentage = duty_cycle * 100  # Convert decimal to percentage
    print(f"➡️ Throttle at {percentage:.1f}% forward")  # Show forward throttle percentage
else:  # Throttle is backward (negative)
    percentage = abs(duty_cycle) * 100  # Convert to positive percentage
    print(f"⬅️ Throttle at {percentage:.1f}% backward")  # Show backward throttle percentage

# Get the battery voltage
voltage = vesc.get_input_voltage()

print(f"Battery Voltage: {voltage} V")

if voltage is None:
    print("❓ Cannot read voltage - check connections")
elif voltage < 11:
    print("🔴 Low battery! Time to charge")
elif voltage < 13:
    print("🟡 Battery getting low")
else:
    print("🟢 Battery level good")
    
print("💡 Voltage is the one reading that should always work - it shows real battery level!")

In [ ]:
# Get the battery voltage
voltage = vesc.get_input_voltage()  # Ask VESC for battery voltage

print(f"Battery Voltage: {voltage} V")  # Display voltage in Volts

if voltage is None:  # Check if voltage reading failed
    print("❓ Cannot read voltage - check connections")  # Connection problem
elif voltage < 11:  # Check if battery is very low
    print("🔴 Low battery! Time to charge")  # Battery needs charging soon
elif voltage < 13:  # Check if battery is getting low
    print("🟡 Battery getting low")  # Battery will need charging
else:  # Battery voltage is good
    print("🟢 Battery level good")  # Battery has plenty of charge

# Get how much electricity is being drawn from the battery
input_current = vesc.get_input_current()

print(f"Battery Current: {input_current} A")

if input_current == 0:
    print("🔋 No power being used from battery")
    print("💡 This will be 0 unless the motor is actively running")
elif input_current > 0:
    print(f"⚡ Drawing {input_current} A from battery")
else:
    print(f"🔌 Putting {abs(input_current)} A back into battery (regenerative braking!)")

In [ ]:
# Get how much electricity is being drawn from the battery
input_current = vesc.get_input_current()  # Ask VESC for battery current

print(f"Battery Current: {input_current} A")  # Display current in Amperes

if input_current == 0:  # Check if no power is being used
    print("🔋 No power being used from battery")  # Battery not being drained
elif input_current > 0:  # Check if power is being used from battery
    print(f"⚡ Drawing {input_current} A from battery")  # Battery is being drained
else:  # Negative current means power going back to battery
    print(f"🔌 Putting {abs(input_current)} A back into battery (regenerative braking!)")  # Power being recovered

### 6. Temperature Monitoring

Just like your phone can get hot when working hard, motors and controllers heat up too. We need to watch the temperature so nothing breaks!

**FET Temperature** = Temperature of the electronic switches in the VESC
**Motor Temperature** = Temperature of the motor itself

In [ ]:
# Get controller temperature (FET = electronic switches)
controller_temp = vesc.get_fet_temperature()  # Ask VESC for controller temperature
motor_temp = vesc.get_motor_temperature()  # Ask VESC for motor temperature

print(f"Controller Temperature: {controller_temp}°C")  # Display controller temp in Celsius
print(f"Motor Temperature: {motor_temp}°C")  # Display motor temp in Celsius

# Check if temperatures are safe
def check_temperature(temp, name):  # Define function to check if temperature is safe
    if temp is None:  # Check if temperature reading failed
        print(f"❓ Cannot read {name} temperature")  # Unable to get temperature
    elif temp < 40:  # Check if temperature is cool
        print(f"❄️ {name} is cool ({temp}°C)")  # Temperature is low and safe
    elif temp < 60:  # Check if temperature is warm but normal
        print(f"🌡️ {name} is warm ({temp}°C) - normal")  # Temperature is normal operating range
    elif temp < 80:  # Check if temperature is getting hot
        print(f"🔥 {name} is getting hot ({temp}°C) - be careful")  # Temperature is high, monitor closely
    else:  # Temperature is dangerously hot
        print(f"🚨 {name} is too hot ({temp}°C) - STOP!")  # Temperature is too high, stop immediately

check_temperature(controller_temp, "Controller")  # Check if controller temperature is safe
check_temperature(motor_temp, "Motor")  # Check if motor temperature is safe

### 7. Energy Usage Tracking

These functions tell us how much energy has been used over time, like a trip meter in a car.

**Amp Hours (Ah)** = How much current used over time (like fuel consumed)
**Watt Hours (Wh)** = How much power used over time (more accurate energy measurement)

In [ ]:
# Get energy usage information
amp_hours_used = vesc.get_amp_hours_consumed()  # Ask VESC how much current used over time
amp_hours_recovered = vesc.get_amp_hours_charged()  # Ask VESC how much current recovered
watt_hours_used = vesc.get_watt_hours_consumed()  # Ask VESC how much power used over time
watt_hours_recovered = vesc.get_watt_hours_charged()  # Ask VESC how much power recovered

print("=== Energy Usage ===")  # Print section header
print(f"Energy Used: {amp_hours_used} Ah, {watt_hours_used} Wh")  # Display energy consumed
print(f"Energy Recovered: {amp_hours_recovered} Ah, {watt_hours_recovered} Wh")  # Display energy recovered

# Calculate efficiency
if amp_hours_used and amp_hours_used > 0:  # Check if we have valid usage data
    recovery_percent = (amp_hours_recovered / amp_hours_used) * 100  # Calculate recovery percentage
    print(f"🔋 Energy recovery: {recovery_percent:.1f}%")  # Display recovery percentage
    
    if recovery_percent > 10:  # Check if recovery is significant
        print("♻️ Good energy recovery! (from braking)")  # High recovery, good efficiency
    else:  # Low recovery is normal for driving
        print("⚡ Mostly using energy (normal for driving)")  # Low recovery is expected

### 8. Advanced Sensors

The VESC has several other sensors that provide additional information:

**Tachometer** = Counts total motor rotations (like an odometer)
**PID Position** = Precise position control information
**ADC Voltages** = Extra analog sensors you can connect
**Servo Value** = Radio control input (like from a remote control)

**📝 Note:** In our setup, the advanced sensors (ADC, servo) are not connected to anything, so they will show 0. The tachometer and PID position will only show interesting values when the motor moves.

In [ ]:
# Get advanced sensor information
tachometer = vesc.get_tachometer_value()  # Ask VESC for total rotation count
pid_position = vesc.get_pid_position()  # Ask VESC for precise position control info
adc_ext = vesc.get_adc_voltage_ext()  # Ask VESC for extra analog sensor 1 voltage
adc_ext2 = vesc.get_adc_voltage_ext2()  # Ask VESC for extra analog sensor 2 voltage
adc_ext3 = vesc.get_adc_voltage_ext3()  # Ask VESC for extra analog sensor 3 voltage
servo_value = vesc.get_servo_value()  # Ask VESC for remote control input value

print("=== Advanced Sensors ===")  # Print section header
print(f"Total Rotations: {tachometer}")  # Display total rotations (like odometer)
print(f"PID Position: {pid_position}")  # Display precise position information
print(f"Extra Sensor 1: {adc_ext} V")  # Display extra sensor 1 voltage
print(f"Extra Sensor 2: {adc_ext2} V")  # Display extra sensor 2 voltage
print(f"Extra Sensor 3: {adc_ext3} V")  # Display extra sensor 3 voltage
print(f"Remote Control: {servo_value}")  # Display remote control input

print("\n💡 Note: ADC and servo sensors show 0 because nothing is connected to them")  # Explain why sensors show 0
print("   Tachometer counts motor rotations - spin the motor to see it change!")  # Explain how to see tachometer change

# Explain tachometer
if tachometer:  # Check if tachometer has a value
    if abs(tachometer) > 1000:  # Check if motor has spun many times
        print(f"🔄 Motor has spun {abs(tachometer)} times total!")  # Motor has been used a lot
    else:  # Motor hasn't spun much
        print(f"🔄 Motor has spun {abs(tachometer)} times (just getting started)")  # Motor is lightly used
else:  # Tachometer is zero
    print("🔄 Tachometer is 0 - motor hasn't moved yet")  # Motor hasn't moved since reset

### 9. Get All Information at Once

Instead of calling each function separately, we can get all the information at once with `get_all_telemetry()`. This is faster and gives us everything organized in groups.

## Part 2: Sending Commands to the VESC (Motor Control)

Now that you know how to **read information from** the VESC, let's learn how to **send commands to** the VESC to control the motor!

⚠️ **SAFETY FIRST!** ⚠️

This is where we go from just "listening" to the VESC to actually "talking" to it and making things move! Always make sure:
1. Nothing is connected to the motor that could get hurt
2. The motor is securely mounted
3. You have a way to quickly stop if something goes wrong
4. Start with very small values and work your way up

## Part 2: Controlling the Motor (Write Functions)

⚠️ **SAFETY FIRST!** ⚠️

Now we'll learn how to control the motor. **This is where things can move!** Always make sure:
1. Nothing is connected to the motor that could get hurt
2. The motor is securely mounted
3. You have a way to quickly stop if something goes wrong
4. Start with very small values and work your way up

### Safety Check

Before we can control the motor, let's enable motor control. **Only change this to `True` when it's safe!**

In [ ]:
# SAFETY: Set this to True only when it's safe to run the motor
MOTOR_CONTROL_ENABLED = False  # Safety flag to prevent accidental motor movement

if MOTOR_CONTROL_ENABLED:  # Check if motor control is enabled
    print("🚨 MOTOR CONTROL ENABLED - Motor can now move!")  # Warn that motor can move
    print("⚠️ Make sure it's safe before running the next cells!")  # Remind user to be safe
else:  # Motor control is disabled for safety
    print("🛑 Motor control is DISABLED for safety")  # Confirm motor won't move
    print("   Change MOTOR_CONTROL_ENABLED = True to enable motor commands")  # Tell user how to enable
    print("   Only do this when it's safe for the motor to move!")  # Emphasize safety

### 1. Duty Cycle Control (Speed Control)

**Duty cycle control** is like a throttle or gas pedal. It controls how much power goes to the motor.

- `vesc.set_duty_cycle(0.1)` = 10% throttle (slow)
- `vesc.set_duty_cycle(0.5)` = 50% throttle (medium)
- `vesc.set_duty_cycle(1.0)` = 100% throttle (full speed)
- `vesc.set_duty_cycle(-0.3)` = 30% throttle backwards
- `vesc.set_duty_cycle(0)` = STOP

In [ ]:
if MOTOR_CONTROL_ENABLED:  # Check if motor control is safe to use
    print("Testing duty cycle control...")  # Tell user what we're doing
    
    # Start with a very small throttle (5%)
    print("Setting throttle to 5%...")  # Tell user we're starting motor
    success = vesc.set_duty_cycle(0.05)  # Send 5% throttle command to VESC
    
    if success:  # Check if command was sent successfully
        print("✅ Command sent successfully!")  # Confirm command worked
        
        # Wait a moment and check what happened
        time.sleep(2)  # Wait 2 seconds for motor to respond
        rpm = vesc.get_rpm()  # Ask VESC for current motor speed
        current = vesc.get_motor_current()  # Ask VESC for current motor current
        
        print(f"Motor is now spinning at {rpm} RPM")  # Show new motor speed
        print(f"Using {current} A of current")  # Show how much electricity motor is using
        
        # Stop the motor
        print("\nStopping motor...")  # Tell user we're stopping motor
        vesc.set_duty_cycle(0)  # Send stop command (0% throttle)
        print("✅ Motor stopped")  # Confirm motor is stopped
    else:  # Command failed to send
        print("❌ Failed to send command")  # Tell user command didn't work
        
else:  # Motor control is disabled
    print("🛑 Motor control disabled. Enable it above to test duty cycle control.")  # Tell user control is off
    print("   Example: vesc.set_duty_cycle(0.1) would set 10% throttle")  # Show example command

### 2. Current Control (Torque Control)

**Current control** is like controlling how hard the motor pushes, regardless of speed.

Think of it like this:
- Duty cycle = "How fast do you want to go?"
- Current = "How hard do you want to push?"

Current control is better when you need precise force, like lifting something heavy or climbing a hill.

In [ ]:
if MOTOR_CONTROL_ENABLED:  # Check if motor control is safe to use
    print("Testing current control...")  # Tell user what we're doing
    
    # Set a small current (2 Amperes)
    print("Setting current to 2A (gentle push)...")  # Tell user we're setting current
    success = vesc.set_current(2.0)  # Send 2 Ampere current command to VESC
    
    if success:  # Check if command was sent successfully
        print("✅ Command sent successfully!")  # Confirm command worked
        
        # Wait and see what happens
        time.sleep(2)  # Wait 2 seconds for motor to respond
        rpm = vesc.get_rpm()  # Ask VESC for current motor speed
        actual_current = vesc.get_motor_current()  # Ask VESC for actual current being used
        
        print(f"Motor RPM: {rpm}")  # Show motor speed
        print(f"Actual current: {actual_current} A")  # Show actual current
        print("Notice: With current control, the motor tries to maintain 2A")  # Explain current control
        print("        even if something tries to slow it down!")  # Explain how current control works
        
        # Stop the motor
        print("\nStopping motor...")  # Tell user we're stopping motor
        vesc.set_current(0)  # Send stop command (0 Amperes)
        print("✅ Motor stopped")  # Confirm motor is stopped
    else:  # Command failed to send
        print("❌ Failed to send command")  # Tell user command didn't work
        
else:  # Motor control is disabled
    print("🛑 Motor control disabled. Enable it above to test current control.")  # Tell user control is off
    print("   Example: vesc.set_current(5.0) would set 5A of current")  # Show example command

### 3. Brake Control (Regenerative Braking)

**Brake current** makes the motor act like a generator, slowing it down while putting energy back into the battery!

This is like:
- A dynamo on a bicycle (creates resistance and generates electricity)
- Regenerative braking in hybrid cars
- Going downhill and charging your battery at the same time

The motor will try to slow down and generate the amount of current you specify.

In [ ]:
if MOTOR_CONTROL_ENABLED:  # Check if motor control is safe to use
    print("Testing regenerative braking...")  # Tell user what we're doing
    
    # First, let's spin up the motor
    print("Step 1: Spinning up motor to 20% throttle...")  # Tell user we're starting motor
    vesc.set_duty_cycle(0.2)  # Send 20% throttle command to VESC
    time.sleep(3)  # Let it spin up for 3 seconds
    
    initial_rpm = vesc.get_rpm()  # Ask VESC for current motor speed
    print(f"Motor is spinning at {initial_rpm} RPM")  # Show motor speed
    
    # Now apply regenerative braking
    print("\nStep 2: Applying regenerative brake (2A)...")  # Tell user we're braking
    vesc.set_brake_current(2.0)  # Send brake command with 2 Amperes
    
    print("Watch how the motor slows down while generating electricity:")  # Explain what will happen
    
    # Monitor the braking process
    for i in range(8):  # Loop 8 times to monitor braking
        time.sleep(0.5)  # Wait half a second between readings
        rpm = vesc.get_rpm()  # Ask VESC for current motor speed
        current = vesc.get_motor_current()  # Ask VESC for current motor current
        
        print(f"  {i*0.5+0.5}s: {rpm} RPM, {current} A")  # Show time, speed, and current
        
        if abs(rpm or 0) < 100:  # Check if motor has nearly stopped (speed less than 100 RPM)
            print("  Motor has slowed down significantly!")  # Tell user motor is nearly stopped
            break  # Exit the monitoring loop early
    
    # Stop braking
    print("\nStep 3: Stopping brake...")  # Tell user we're stopping the brake
    vesc.set_brake_current(0)  # Send stop brake command (0 Amperes)
    print("✅ Regenerative braking test complete!")  # Confirm test is done
    print("🔋 The motor acted like a generator and put energy back into the battery")  # Explain what happened
    
else:  # Motor control is disabled
    print("🛑 Motor control disabled. Enable it above to test regenerative braking.")  # Tell user control is off
    print("   Example: vesc.set_brake_current(3.0) would brake with 3A of current")  # Show example command
    print("   This makes the motor slow down and generate electricity!")  # Explain what brake command does

### 4. Emergency Stop Function

It's always good to have a quick way to stop everything!

In [ ]:
# Emergency stop - sets everything to zero
print("🚨 EMERGENCY STOP - Stopping all motor activity")  # Tell user we're stopping everything

vesc.set_duty_cycle(0)      # Stop throttle (set to 0%)
vesc.set_current(0)         # Stop current (set to 0 Amperes)
vesc.set_brake_current(0)   # Stop braking (set to 0 Amperes)

# Check that everything stopped
time.sleep(1)  # Wait 1 second for commands to take effect
rpm = vesc.get_rpm()  # Ask VESC for motor speed
current = vesc.get_motor_current()  # Ask VESC for motor current
duty = vesc.get_duty_cycle()  # Ask VESC for duty cycle

print(f"Final status:")  # Print status header
print(f"  RPM: {rpm}")  # Show final motor speed
print(f"  Current: {current} A")  # Show final motor current
print(f"  Duty Cycle: {duty}")  # Show final duty cycle
print("✅ All motor commands set to zero - safe state")  # Confirm everything is safe

## Part 3: Summary and What You've Learned

Congratulations! You now know how to:

### 📊 **Read Information (Telemetry)**
- `get_rpm()` - Motor speed (how fast it spins)
- `get_motor_current()` - How much electricity the motor uses
- `get_duty_cycle()` - Current throttle setting (like gas pedal position)
- `get_input_voltage()` - Battery voltage
- `get_input_current()` - How much electricity is taken from battery
- `get_fet_temperature()` - Controller temperature
- `get_motor_temperature()` - Motor temperature
- `get_amp_hours_consumed()` - Energy used over time
- `get_amp_hours_charged()` - Energy recovered over time
- `get_watt_hours_consumed()` - Power used over time
- `get_watt_hours_charged()` - Power recovered over time
- `get_tachometer_value()` - Total rotations
- `get_pid_position()` - Precise position
- `get_adc_voltage_ext()` - Extra sensor 1
- `get_adc_voltage_ext2()` - Extra sensor 2
- `get_adc_voltage_ext3()` - Extra sensor 3
- `get_servo_value()` - Remote control input
- `get_all_telemetry()` - Everything at once!

### 🎮 **Control the Motor**
- `set_duty_cycle(value)` - Speed control (like gas pedal) -1.0 to 1.0
- `set_current(value)` - Force control (how hard to push) in Amperes
- `set_brake_current(value)` - Regenerative braking (slow down + charge battery)

### 🧠 **Key Concepts You Learned**
1. **RPM** = Revolutions Per Minute (speed)
2. **Current** = Electricity flow (Amperes)
3. **Voltage** = Electrical pressure (Volts)
4. **Duty Cycle** = Percentage of power (like throttle)
5. **Regenerative Braking** = Motor acts like generator
6. **Temperature Monitoring** = Keep things from overheating
7. **Energy Tracking** = Monitor consumption and recovery

### 🛡️ **Safety Rules**
1. Always check temperatures
2. Start with small values
3. Have an emergency stop ready
4. Monitor battery voltage
5. Secure the motor before testing

You're now ready to build awesome electric motor projects! 🚀

## Part 3: Summary and What You've Learned

Congratulations! You now know how to **communicate with a VESC**! You can both listen to it and talk to it:

### 📊 **Reading Information from VESC (Listening)**
- `get_rpm()` - Motor speed (how fast it spins)
- `get_motor_current()` - How much electricity the motor uses
- `get_duty_cycle()` - Current throttle setting (like gas pedal position)
- `get_input_voltage()` - Battery voltage *(always works!)*
- `get_input_current()` - How much electricity is taken from battery
- `get_fet_temperature()` - Controller temperature
- `get_motor_temperature()` - Motor temperature
- `get_amp_hours_consumed()` - Energy used over time
- `get_amp_hours_charged()` - Energy recovered over time
- `get_watt_hours_consumed()` - Power used over time
- `get_watt_hours_charged()` - Power recovered over time
- `get_tachometer_value()` - Total rotations *(spin motor to see changes)*
- `get_pid_position()` - Precise position *(changes when motor moves)*
- `get_adc_voltage_ext()` - Extra sensor 1 *(not connected - shows 0)*
- `get_adc_voltage_ext2()` - Extra sensor 2 *(not connected - shows 0)*
- `get_adc_voltage_ext3()` - Extra sensor 3 *(not connected - shows 0)*
- `get_servo_value()` - Remote control input *(not connected - shows 0)*
- `get_all_telemetry()` - Everything at once!

### 🎮 **Sending Commands to VESC (Talking)**
- `set_duty_cycle(value)` - Speed control (like gas pedal) -1.0 to 1.0
- `set_current(value)` - Force control (how hard to push) in Amperes
- `set_brake_current(value)` - Regenerative braking (slow down + charge battery)

### 🧠 **Key Communication Concepts You Learned**
1. **Telemetry** = Data the VESC sends to us
2. **Commands** = Instructions we send to the VESC
3. **Real-time Communication** = Talking back and forth continuously
4. **Data Only Shows When Active** = Most readings are 0 when motor isn't moving
5. **Voltage Always Works** = Battery voltage is always available
6. **Manual Testing** = Spin motor by hand to see data changes
7. **Safety in Communication** = Always start with small, safe commands

### 🛡️ **Communication Safety Rules**
1. Always read before commanding
2. Start with small command values
3. Monitor the VESC's responses
4. Use emergency stop when needed
5. Understand what each data value means

You're now ready to build awesome projects that **talk to VESCs**! 🚀📡